In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math

# Zadanie 1

In [1]:
# Parametry problemu
demand_per_month = 3
months = 6
max_prod = 5
max_inv = 4

# Koszty: koszt uruchomienia oraz tabela kosztów zmiennych dla produkcji 0..5 szt.
fixed_cost = 13  # tys. PLN za uruchomienie produkcji (jeśli x > 0)
prod_cost = {0: 0, 1: 16, 2: 18, 3: 19, 4: 20, 5: 24}  # zmienny koszt produkcji x szt. (tys. PLN)

# Inicjalizacja tablic DP (słowniki dla każdego miesiąca)
dp = [dict() for _ in range(months + 2)]      # dp[t][inv] = minimalny koszt od miesiąca t przy zapasie inv na początku
decision = [dict() for _ in range(months + 2)]  # decyzja (produkcja x) dla stanu (t, inv) w optymalnym planie

# Warunek końcowy dla t = 7 (po ostatnim miesiącu)
for inv in range(max_inv + 1):
    dp[months+1][inv] = 0 if inv == 0 else float('inf')

# Obliczenia dynamiczne wstecz (od miesięcy 6 do 1)
for t in range(months, 0, -1):
    for inv in range(max_inv + 1):
        best_cost = float('inf')
        best_x = 0
        # Przeszukanie decyzji x = 0..5
        for x in range(max_prod + 1):
            inv_end = inv + x - demand_per_month  # zapas na koniec miesiąca t
            # Sprawdzenie ograniczeń: zaspokojenie popytu i pojemność magazynu
            if inv_end < 0 or inv_end > max_inv:
                continue
            # Koszt produkcji w miesiącu t (uwzględnia koszt stały jeśli x > 0)
            cost_prod = (fixed_cost + prod_cost[x]) if x > 0 else 0
            # Koszt magazynowania na koniec miesiąca t (inv_end jednostek * 2 tys. PLN)
            cost_hold = inv_end * 2
            # Całkowity koszt (bieżący + przyszły)
            total_cost = cost_prod + cost_hold + dp[t+1][inv_end]
            if total_cost < best_cost:
                best_cost = total_cost
                best_x = x
        # Zapisz najlepszy koszt i decyzję dla stanu (t, inv)
        dp[t][inv] = best_cost
        decision[t][inv] = best_x

# Rekonstrukcja optymalnego planu od t=1, początkowy zapas 0
inv = 0
plan = []
for t in range(1, months + 1):
    x = decision[t][inv]
    inv_end = inv + x - demand_per_month
    plan.append((x, inv_end))
    inv = inv_end

# Wynik: minimalny koszt i szczegóły produkcji w każdym miesiącu
print(f"Minimalny całkowity koszt: {dp[1][0]} tys. PLN")
for t, (x, inv_end) in enumerate(plan, start=1):
    print(f"Miesiąc {t}: produkcja = {x}, zapas na koniec = {inv_end}")

Minimalny całkowity koszt: 156 tys. PLN
Miesiąc 1: produkcja = 4, zapas na koniec = 1
Miesiąc 2: produkcja = 5, zapas na koniec = 3
Miesiąc 3: produkcja = 0, zapas na koniec = 0
Miesiąc 4: produkcja = 4, zapas na koniec = 1
Miesiąc 5: produkcja = 5, zapas na koniec = 3
Miesiąc 6: produkcja = 0, zapas na koniec = 0


# Zadanie 2


In [4]:
# Prawdopodobieństwa, że urządzenie nie ulegnie awarii w zależności od kwoty przeznaczonej na remont
probabilities = {
    'U1': [0.2, 0.2, 0.4, 0.6, 0.7],
    'U2': [0.1, 0.3, 0.5, 0.7, 0.8],
    'U3': [0.2, 0.4, 0.6, 0.8, 0.9],
    'U4': [0.1, 0.2, 0.3, 0.5, 0.7]
}

# Wartości dostępne do wydania na remont w tys. PLN
amounts = [0, 1, 2, 3, 4]  # Kwoty przeznaczone na remonty (0-4 tys. PLN)

# Budżet całkowity
total_budget = 10  # 10 tysięcy PLN

# Funkcja do obliczenia maksymalnego prawdopodobieństwa braku awarii oraz przydziału środków
def max_reliability_with_allocation(total_budget, probabilities):
    num_devices = len(probabilities)  # Liczba urządzeń
    
    # Tablica DP: dp[i][j] oznacza maksymalne prawdopodobieństwo dla i urządzeń i j tysięcy PLN
    dp = np.zeros((num_devices + 1, total_budget + 1))
    
    # Tablica przechowująca optymalny przydział środków
    allocation = np.zeros((num_devices + 1, total_budget + 1), dtype=int)
    
    # Inicjalizacja - dla 0 urządzeń prawdopodobieństwo awarii to 1 (nie ma urządzeń do naprawy)
    dp[0, 0] = 1  # Nie ma urządzeń, więc prawdopodobieństwo braku awarii to 1

    # Programowanie dynamiczne
    for device in range(1, num_devices + 1):  # Przechodzimy po urządzeniach
        for budget in range(total_budget + 1):  # Przechodzimy po dostępnych kwotach
            for spent in amounts:
                if budget - spent >= 0:
                    prob_device = probabilities[f'U{device}'][spent]
                    new_prob = dp[device - 1, budget - spent] * prob_device
                    if new_prob > dp[device, budget]:
                        dp[device, budget] = new_prob
                        allocation[device, budget] = spent

    # Maksymalne prawdopodobieństwo braku awarii całego obiektu
    max_probability = dp[num_devices, total_budget]

    # Odtwarzanie optymalnego przydziału środków
    budget_left = total_budget
    optimal_allocation = []
    for device in range(num_devices, 0, -1):
        spent = allocation[device, budget_left]
        optimal_allocation.append((f'U{device}', spent))
        budget_left -= spent
    
    # Zwracamy maksymalne prawdopodobieństwo oraz przydział środków na urządzenia
    optimal_allocation.reverse()
    return max_probability, optimal_allocation

# Obliczamy maksymalne prawdopodobieństwo i przydział środków
max_probability, optimal_allocation = max_reliability_with_allocation(total_budget, probabilities)

# Wyświetlamy wyniki
print(f'Maksymalne prawdopodobieństwo braku awarii: {max_probability:.4f}')
print("Optymalny przydział środków:")
for device, spent in optimal_allocation:
    print(f'{device}: {spent} tys. PLN')

Maksymalne prawdopodobieństwo braku awarii: 0.0900
Optymalny przydział środków:
U1: 3 tys. PLN
U2: 2 tys. PLN
U3: 2 tys. PLN
U4: 3 tys. PLN
